In [0]:
import json
from glob import glob
bronze_inputs=json.loads(dbutils.jobs.taskValues.get(taskKey="job_settings",key="bronze"))
silver_inputs=json.loads(dbutils.jobs.taskValues.get(taskKey="job_settings",key="silver"))
gold_inputs=json.loads(dbutils.jobs.taskValues.get(taskKey="job_settings",key="gold"))
layer_files={
    "bronze":[f for d in bronze_inputs for f in glob(f"./layer_*_bronze/{d['table']}.json")],
    "silver":[f for d in silver_inputs for f in glob(f"./layer_*_silver/{d['table']}.json")],
    "gold":[f for d in gold_inputs for f in glob(f"./layer_*_gold/{d['table']}.json")]
}

In [0]:
import json
from pyspark.sql.types import StructType
layers=["bronze","silver","gold"]
required={
    "bronze":["read_function","transform_function","write_function","dst_table_name","file_schema"],
    "silver":["read_function","transform_function","write_function","src_table_name","dst_table_name","composite_key","business_key"],
    "gold":["read_function","transform_function","write_function","src_table_name","dst_table_name","composite_key","business_key"]
}
errs=[]
for layer in layers:
    for path in layer_files[layer]:
        cfg=json.loads(open(path).read())
        for k in required[layer]:
            if k not in cfg:
                errs.append(f"{path} missing {k}")
for layer in layers:
    for path in layer_files[layer]:
        cfg=json.loads(open(path).read())
        dst=cfg["dst_table_name"]
        if not spark.catalog.tableExists(dst):
            errs.append(f"{layer} table {dst} not found")
if errs:
    raise RuntimeError("Sanity check failed: "+", ".join(errs))
else:
    print("Sanity check passed")